In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
df = pd.read_csv('insurance_data.csv')
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df["bought_insurance"], test_size=0.2, random_state=25)

In [ ]:
# scaling
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100


## Simple Model (with Tensorflow)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(1, activation="sigmoid", input_shape=(2,), 
                      bias_initializer='zero', # bias = 0
                       kernel_initializer='one' # weights = 1
                      )
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train, epochs=5000)



In [ ]:
model.evaluate(X_test_scaled, y_test)


In [ ]:
model.predict(X_test_scaled)

In [ ]:
# get weights

coef, intercept = model.get_weights()
coef, intercept

## Simple Predictor (from scratch)

In [ ]:
import math
def sigmoid(x):
    return 1 / (1+math.exp(-x))

In [ ]:
def prediction_function(age, affordability):
    weighted_sum = coef[0]*age + coef[1]*affordability + intercept
    return sigmoid(weighted_sum)

prediction_function(0.47, 1)

## Simple Gradient Descent (from scratch)

In [ ]:
# binary_crossentropy
def log_loss(y_true, y_predicted):
    epsilon=1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1-epsilon) for i in y_predicted]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true)*np.log(1-y_predicted_new))

In [ ]:
def sigmoid_np(x):
    return 1/(1+np.exp(-x))

sigmoid_np(np.array([12,0,1]))

In [ ]:
def gradient_descent(age, affordibility, y_true, epochs, loss_treshold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    
    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + bias
        y_predicted = sigmoid_np(weighted_sum)
        loss = log_loss(y_true, y_predicted)
        
        w1d = (1/n)*np.dot(np.transpose(age), (y_predicted - y_true))
        w2d = (1/n)*np.dot(np.transpose(affordibility), (y_predicted - y_true))
        
        bias_d = np.mean(y_predicted-y_true)
        
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d
        
        print(f'Epoch: {i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
        
        if loss<=loss_treshold:
            break
    
    return w1, w2, bias

In [ ]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, epochs=2000, loss_treshold=0.4631)

## Simple Neural Network (from scratch)

In [ ]:
class MyNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_treshold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_treshold)
        
    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid_np(weighted_sum)
    
    def gradient_descent(self, age, affordibility, y_true, epochs, loss_treshold):
        w1 = self.w1
        w2 = self.w2
        bias = self.bias
        rate = 0.5
        n = len(age)

        for i in range(epochs):
            weighted_sum = w1*age + w2*affordibility + bias
            y_predicted = sigmoid_np(weighted_sum)
            loss = log_loss(y_true, y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age), (y_predicted - y_true))
            w2d = (1/n)*np.dot(np.transpose(affordibility), (y_predicted - y_true))

            bias_d = np.mean(y_predicted-y_true)

            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            if i%50==0:
                print(f'Epoch: {i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

            if loss<=loss_treshold:
                print(f'Break! Final -- Epoch: {i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias

  

In [ ]:
custom_model = MyNN()
custom_model.fit(X_train_scaled, y_train, epochs=500, loss_treshold=0.4631)

In [ ]:
custom_model.predict(X_test_scaled)

In [ ]:
model.predict(X_test_scaled)